

# Create an interface with ZSL and relax it using EMT potentials

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Drop the materials files into the "uploads" folder in the JupyterLab file browser
1. Set Input Parameters (e.g. `distance_z`, `max_area`, `miller_indices`) below or use the default values
1. Click "Run" > "Run All" to run all cells
1. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
1. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials
Imported `InterfaceSettings` is a class that specifies the parameters for the construction of the interface. The default values are assumed if properties are not set during the initialization.
Additionally, specify if the termination selection is done using interactive prompt, or the via selecting the termination index in the code.

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceSettings

interface_settings = InterfaceSettings()
interface_settings.max_area = 50  # maximum area to consider when matching
interface_settings.SubstrateParameters.miller_indices = (1, 1, 1) # the Miller indices of the interfacial plane of the substrate
interface_settings.SubstrateParameters.thickness = 6 # substrate thickness in layers
interface_settings.LayerParameters.miller_indices = (0, 0, 1)  # the Miller indices of the interfacial plane of the layer
interface_settings.LayerParameters.thickness = 1 # layer thickness in layers

IS_TERMINATION_SELECTION_INTERACTIVE = False  # if True, the user can select the termination interactively
TERMINATION_INDEX = 0  # the default termination index that is used if no termination selected, ignored in interactive mode

### 1.2. Set Algorithm Parameters for Strain Matching (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLParameters
interface_settings.ZSLParameters = ZSLParameters(
    max_area_tol=0.09,  # maximum tolerance on ratio of super-lattices to consider equal
    max_length_tol=0.03,  # maximum length tolerance for two vectors to be considered equal
    max_angle_tol=0.01, # maximum angle tolerance for two sets of vectors to have equal angles
)

### 1.3. Set Relaxation Parameters

In [ ]:
RELAXATION_PARAMETERS = {
    "FMAX": 0.018,
}

from mat3ra.made.tools.modify import RelaxationSettings, CalculatorEnum, OptimizerEnum
relaxation_settings = RelaxationSettings()

relaxation_settings.optimizer = OptimizerEnum.BFGS
relaxation_settings.calculator = CalculatorEnum.EMT
relaxation_settings.fmax = 0.05

## 2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` as directed in README.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

## 3. Load input Materials


In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

from utils.visualize import visualize_materials as visualize

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
visualize(materials, repetitions=[1, 1, 1], rotation="0x")

## 4. Create interfaces

### 4.1. Initialize the interface builder

Initialize the interface builder with the materials and interface settings.

In [ ]:
from mat3ra.made.tools.build import init_interface_builder

interface_builder = init_interface_builder(
    substrate=materials[0],
    layer=materials[1],
    settings=interface_settings
)

### 4.2. Select the termination
Possible terminations for the interface are found by the interface builder. The user can select the termination interactively or use the default one.

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide
terminations = interface_builder.terminations

if IS_TERMINATION_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        selected_termination = await ui_prompt_select_array_element_by_index_pyodide(terminations, element_name="termination")
    else:
         selected_termination = ui_prompt_select_array_element_by_index(terminations, element_name="termination")
else:    
    selected_termination = terminations[TERMINATION_INDEX]

### 4.3. Create interfaces for the selected termination

In [ ]:
from mat3ra.made.tools.build import create_interfaces

interface_data_holder = create_interfaces(
    settings=interface_settings,
    sort_by_strain_and_size=True,
    remove_duplicates=True,
    interface_builder=interface_builder,
    termination=selected_termination,
)

### 4.3. Print out interface with the lowest strain for selected termination


In [ ]:
print(f"Interface with lowest strain for termination {selected_termination} (index 0):")
interfaces = interface_data_holder.get_interfaces_for_termination(selected_termination)
first_interface = interfaces[0]
print(f"    strain: {first_interface.get_mean_abs_strain() * 100:.3f}%")
print("    number of atoms:", first_interface.num_sites)

## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.

In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interface_data_holder, PLOT_SETTINGS)

print("Terminations: \n", interface_data_holder.terminations)

## 6. Select the interface to relax

### 6.1. Select the interface with the desired termination and strain


Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [ ]:
# Could be either the termination as tuple, e.g. `('Ni_P6/mmm_1', 'C_C2/m_2')` or its index: `0`
termination_or_its_index = selected_termination
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = 0
interface = interface_data_holder.get_interfaces_as_materials(termination_or_its_index, interfaces_slice_range_or_index)[0]

### 6.2. Visualize the selected interface(s)

In [ ]:
visualize(interface, repetitions=[1, 1, 1], rotation="0x")

## 7. Apply relaxation
### 7.1. Apply relaxation to the selected interface

In [ ]:
from utils.plot import create_realtime_plot, update_plot

from mat3ra.made.tools.modify import relax_atoms
from mat3ra.made.tools.convert import to_ase

final_interface = relax_atoms(Material(interface), relaxation_settings)

f = create_realtime_plot()
steps = []
energies = []
update_plot(f, steps, energies)

visualize(final_interface, repetitions=[1, 1, 1], rotation="0x")

In [ ]:
import plotly.graph_objs as go
from IPython.display import display
from plotly.subplots import make_subplots
from src.utils import ase_to_poscar, pymatgen_to_ase
from ase.optimize import BFGS
from ase.calculators.emt import EMT


# Set up the calculator 
calculator = EMT()
print(calculator)

# Set up the interface for relaxation
ase_interface = to_ase(interface)
print(ase_interface)
ase_interface.set_calculator(calculator)
print(ase_interface)
dyn = BFGS(ase_interface)



In [ ]:
# Initialize empty lists to store steps and energies
steps = []
energies = []

# Create a plotly figure widget
fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter"}]])
scatter = go.Scatter(x=[], y=[], mode='lines+markers', name='Energy')
fig.add_trace(scatter)
fig.update_layout(title_text='Real-time Optimization Progress', xaxis_title='Step', yaxis_title='Energy (eV)')

# Display figure widget
f = go.FigureWidget(fig)
display(f)


# Define a callback function to update the plot at each step
def plotly_callback():
    step = dyn.nsteps
    energy = ase_interface.get_total_energy()

    # Add the new step and energy to the lists
    steps.append(step)
    energies.append(energy)

    print(f"Step: {step}, Energy: {energy:.4f} eV")

    # Update the figure with the new data
    with f.batch_update():
        f.data[0].x = steps
        f.data[0].y = energies


# Run the relaxation
dyn.attach(plotly_callback, interval=1)
dyn.run(fmax=RELAXATION_PARAMETERS["FMAX"])

# Extract results
ase_original_interface = pymatgen_to_ase(interface)
ase_original_interface.set_calculator(calculator)
ase_final_interface = ase_interface

original_energy = ase_original_interface.get_total_energy()
relaxed_energy = ase_interface.get_total_energy()

# Print out the final relaxed structure and energy
print('Original structure:\n', ase_to_poscar(ase_original_interface))
print('\nRelaxed structure:\n', ase_to_poscar(ase_final_interface))
print(f"The final energy is {float(relaxed_energy):.3f} eV.")

### 7.2. View structure before and after relaxation

In [ ]:
import base64
from ase.io import write
from ase.build import make_supercell
from IPython.display import HTML
import io


def visualize_material_base64(material, title: str, rotation: str = '0x', number_of_repetitions: int = 3):
    """
    Returns an HTML string with a Base64-encoded image for visualization,
    including the name of the file, positioned horizontally.
    """
    # Set the number of unit cell repetition for the structure
    n = number_of_repetitions
    material_repeat = make_supercell(material, [[n, 0, 0], [0, n, 0], [0, 0, 1]])
    text = f"{material.symbols} - {title}"

    # Write image to a buffer to display in HTML
    buf = io.BytesIO()
    write(buf, material_repeat, format='png', rotation=rotation)
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    html_str = f'''
    <div style="display: inline-block; margin: 10px; vertical-align: top;">
        <p>{text}</p>
        <img src="data:image/png;base64,{img_str}" alt="{title}" />
    </div>
    '''
    return html_str


html_original = visualize_material_base64(ase_original_interface, "original", "-90x")
html_relaxed = visualize_material_base64(ase_final_interface, "relaxed", "-90x")

# Display the interfaces before and after relaxation
html_content = f'<div style="display: flex;">{html_original}{html_relaxed}</div>'
display(HTML(html_content))



### 7.3. Calculate energy energy using ASE EMT

In [ ]:
def filter_atoms_by_tag(atoms, material_index):
    """Filter atoms by their tag, corresponding to the material index."""
    return atoms[atoms.get_tags() == material_index]


def calculate_energy(atoms, calculator):
    """Set calculator for atoms and return their total energy."""
    atoms.set_calculator(calculator)
    return atoms.get_total_energy()


def calculate_delta_energy(total_energy, *component_energies):
    """Calculate the delta energy by subtracting component energies from the total energy."""
    return total_energy - sum(component_energies)


# Filter atoms for original and relaxed interfaces
substrate_original_interface = filter_atoms_by_tag(ase_original_interface, SUBSTRATE_PARAMETERS["MATERIAL_INDEX"])
layer_original_interface = filter_atoms_by_tag(ase_original_interface, LAYER_PARAMETERS["MATERIAL_INDEX"])
substrate_relaxed_interface = filter_atoms_by_tag(ase_final_interface, SUBSTRATE_PARAMETERS["MATERIAL_INDEX"])
layer_relaxed_interface = filter_atoms_by_tag(ase_final_interface, LAYER_PARAMETERS["MATERIAL_INDEX"])

# Calculate energies
original_substrate_energy = calculate_energy(substrate_original_interface, calculator)
original_layer_energy = calculate_energy(layer_original_interface, calculator)
relaxed_substrate_energy = calculate_energy(substrate_relaxed_interface, calculator)
relaxed_layer_energy = calculate_energy(layer_relaxed_interface, calculator)

# Calculate delta energies
delta_original = calculate_delta_energy(original_energy, original_substrate_energy, original_layer_energy)
delta_relaxed = calculate_delta_energy(relaxed_energy, relaxed_substrate_energy, relaxed_layer_energy)

# Calculate area and effective delta per area
area = ase_original_interface.get_volume() / ase_original_interface.cell[2, 2]
number_of_interface_atoms = ase_final_interface.get_global_number_of_atoms()
number_of_substrate_atoms = substrate_relaxed_interface.get_global_number_of_atoms()
number_of_layer_atoms = layer_relaxed_interface.get_global_number_of_atoms()
effective_delta_relaxed = (relaxed_energy/number_of_interface_atoms - (relaxed_substrate_energy/number_of_substrate_atoms + relaxed_layer_energy/number_of_layer_atoms)) / (2 * area)

# Print out the metrics
print(f"Original Substrate energy: {original_substrate_energy:.4f} eV")
print(f"Relaxed Substrate energy: {relaxed_substrate_energy:.4f} eV")
print(f"Original Layer energy: {original_layer_energy:.4f} eV")
print(f"Relaxed Layer energy: {relaxed_layer_energy:.4f} eV")
print("\nDelta between interface energy and sum of component energies")
print(f"Original Delta: {delta_original:.4f} eV")
print(f"Relaxed Delta: {delta_relaxed:.4f} eV")
print(f"Original Delta per area: {delta_original / area:.4f} eV/Ang^2")
print(f"Relaxed Delta per area: {delta_relaxed / area:.4f} eV/Ang^2")
print(f"Relaxed interface energy: {relaxed_energy:.4f} eV")
print(f"Effective relaxed Delta per area: {effective_delta_relaxed:.4f} eV/Ang^2 ({effective_delta_relaxed / 0.16:.4f} J/m^2)\n")

# Print out the POSCARs
print("Relaxed interface:\n", ase_to_poscar(ase_final_interface))
print("Relaxed substrate:\n", ase_to_poscar(substrate_relaxed_interface))
print("Relaxed layer:\n", ase_to_poscar(layer_relaxed_interface))